In [52]:
# for loading the csv  file
from google.colab import files
import io
############## general dependencies ####################################
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
#distinct value counter
from collections import Counter


## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

- Split the Data into Training and Testing Sets
- Open the starter code notebook and use it to complete the following steps:

- Read the lending_data.csv data from the Resources folder into a Pandas DataFrame.



In [53]:
data = files.upload()

Saving lending_data.csv to lending_data (1).csv


In [60]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
# Load the data into a Pandas DataFrame
lending_data = pd.read_csv(io.StringIO(data['lending_data.csv'].decode('utf-8')))

# Display sample data
lending_data.head(10)
# Review the DataFrame


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
5,10100.0,7.438,50600,0.407115,4,1,20600,0
6,10300.0,7.490,51100,0.412916,4,1,21100,0
7,8800.0,6.857,45100,0.334812,3,0,15100,0
8,9300.0,7.096,47400,0.367089,3,0,17400,0
9,9700.0,7.248,48800,0.385246,4,0,18800,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

##### NOTE

- A value of 0 in the “loan_status” column means that the loan is healthy. A value of 1 means that the loan has a high risk of defaulting.

- Split the data into training and testing datasets by using train_test_split.

In [61]:
# Separate the data into labels and features

# Separate the y variable, the labels
y =lending_data['loan_status']

# Separate the X variable, the features
X =lending_data.drop(columns=['loan_status'])

In [62]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [63]:
# Review the X variable DataFrame
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [58]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [64]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split and assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [65]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
log_regression = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model= log_regression.fit(X_train,y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [66]:
# Make a prediction using the testing data
testing_data_prediction = lr_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [67]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score (y_test,testing_data_prediction )

0.9520479254722232

In [70]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,testing_data_prediction)

array([[18663,   102],
       [   56,   563]])

### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The predictive precision for the logistic regression model is pretty good at average of 95%. The cells down below will show more precisely how the model is perfoming for specific loan statuses. 

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [74]:
# Print the classification report for the model
print(classification_report(y_test,testing_data_prediction))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



In [76]:
# Import the RandomOverSampler module form imbalanced-learn 
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros_model = RandomOverSampler( random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = ros_model.fit_resample(X_train, y_train)

In [77]:
#Count the distinct values of the resampled labels data
print(sorted(Counter(lending_data['loan_status']).items()))

[(0, 75036), (1, 2500)]


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [78]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
log_reg_model= LogisticRegression(random_state=1)

# Fit the model using the resampled training data
fitted_log_reg_model= log_reg_model.fit(X_train,y_train)

# Make a prediction using the testing data
training_data_predictions= log_reg_model.predict(X_train)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [80]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_train, training_data_predictions)

0.9494259906569136

In [81]:
# Generate a confusion matrix for the model
confusion_matrix(y_train, training_data_predictions)

array([[55994,   277],
       [  181,  1700]])

In [82]:
# Print the classification report for the model
print(classification_report(y_train, training_data_predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56271
           1       0.86      0.90      0.88      1881

    accuracy                           0.99     58152
   macro avg       0.93      0.95      0.94     58152
weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** In terms of accuracy regarding the healthy loans, both models perfom well. However both models have lower accuracy scores for the high risk loans. That being said, it is also important to note that the OverSampled data model's accuracy and precision rate increased by 1% . It is not a significant amount of improvement but it might be helpful when trying to alter a few things to improve perfomance rates.